In [3]:
import pinecone
import os, getpass
import tiktoken

/Users/eaariash/desarrollo/ia_generativa/chat_ia_generative/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [44]:
api_key = getpass.getpass("Ingresa tu API Key de Pinecone : ")
os.environ["PINECONE_API_KEY"] = api_key

In [43]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingresa tu API Key de OpenAI : ")

## Creando un index (tabla) en Pinecone

In [7]:
env = "gcp-starter" #free tier
index_name = "knowledge-base-stable-difussion"
dimension = 1536


pinecone.init(api_key = api_key, environment = env)

pinecone.create_index(
    index_name, 
    dimension = dimension
)

print("Index " + index_name + " creado con éxito en Pinecone")

Index knowledge-base-stable-difussion creado con éxito en Pinecone


In [38]:
from pinecone import Pinecone, ServerlessSpec

index_name= 'idbookmentor'
os.environ["OPENAI_API_KEY"] = index_name

pc = Pinecone(api_key=api_key)
pc.create_index(
    name="idbookmentor",
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-west-2"
    ) 
)

## Estadísticas de un index

In [39]:
index_name='idbookmentor'
index = pc.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


## Lista de index en Pinecone

In [24]:
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'idbookmentor-4g9qltf.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'idbookmentor',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

## Cargando datos (pdf) y generando sus fragmentos

In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("pdf/Como_ganar_amigos_e_influir_sobre_los_demas.pdf")
data = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(documents = data)

In [14]:
len(docs),docs

(272,
 [Document(page_content='Convertido en un clásico de las relaciones humanas, Cómo ganar amigos…\ncontiene princip ios tan sólidos que siendo escritos en 1936, siguen tan\nvigentes como entonces. Aplicando su gran conocimiento de las personas,\ndotes de observación y experiencia profesional, Dale Carnegie consiguió\ncrear un tratado de psicología cotidiana que no ha perdido ni un ápice de\nactualidad y que en buena medida es el origen del marketing  moderno.\nSegún Carnegie, el éxito económico depende en 15 por ciento del\nconocimiento profesional y en un 85 por ciento de la capacidad para\nexpresar ideas, asumir el liderazgo y despertar el entusiasmo en los demás.\nA través de consejos prácticos y ejemplos tomados tanto de personajes\nrelevantes en la historia como de gente común, Carnegie muestra las\ntécnicas fundam entales para gustar a los demás, comprender sus puntos de\nvista e imponer las propias ideas sin causar resentimiento.\nCon más de 15 millones de ejemplares vendido

## Cargando embeddings a Pinecone

In [27]:
from langchain_pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_documents(docs, embeddings, index_name = index_name)

Nos vamos a nuestra cuenta de Pinecone, para validar que se hayan insertados los embeddings

In [29]:
index = pc.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 272}},
 'total_vector_count': 272}


## Consultando datos del index

![Imagen](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/SemanticSearch.png)

## Muestra los 5 embeddings más relevantes con la pregunta

In [31]:
query = "Como mejorar mi caracter y firmeza"
docs = docsearch.similarity_search(query, k = 5) 
docs

[Document(page_content='Uno de los rasgo s más hermosos que ha tenido Franklin, a mi juicio, es\nla forma  en que aceptó esta dolorosa lección. Tenía ya edad suficiente, y\nsuficiente cordu ra, para comprender que era exacta, que si seguía como\nhasta entonces sólo podría llegar al fracaso y a la catástrofe social. Dio,\npues, una media vuelta. Comen zó inmediatamente a modificar su actitud\ninsolente, llena de prejuicios.\n«Adopté la regla…», refiere Franklin en su biografía:\n… de eludir toda contradicción directa de los sentimientos de\nlos demás, y toda afirmación positiva de los míos. Hasta me\nprohibí el empleo de aquellas palabras o expresiones que\nsignifican una opinión fija, como por cierto, indudablemente ,\netc., y adopté, en lugar de ellas, creo, entiendo, o imagino que\nuna cosa es así ; o así me parece por el momento . Cuando otra\npersona asegur aba algo que a mi juicio era un error, yo me\nnegaba el placer de contradecirla abiertamente y de demostrar\nen seguida algún 

In [32]:
query = "Quiero que una persona haga lo que yo le diga y no me contradiga"
docs = docsearch.similarity_search(query, k = 5)
docs

[Document(page_content='Pero no lo hizo: me agradeció mi trabajo y obser vó que no era\ninfrecuente que alguien cometiera un error la primera vez que\nse le asignaba una tarea de impor tancia, y que confiaba en que\nel inform e final sería correcto y útil para la compañía. Me\naseguró, delant e de todos mis colegas, que tenía fe en mí, y\nsabía que yo había puesto lo mejor de mí, y que el motivo de\nesta falla era mi falta de experiencia, no mi falta de capacidad.\nSalí de esa reunión caminando en las nubes, y juré que nunca\ndecepcionaría a ese extraordinario jefe que tenía.\nAún cuando tengamos razón y la otra persona esté claramente equivocada,\nsólo haremos daño si le hacemos perder prestigio.\n \nEl legen dario escritor y pionero de la aviación A. de Saint Exupéry\nescribió:\nNo tengo derecho a decir o hacer nada que disminuya a un hombre\nante sí mismo . Lo que impor ta no es lo que yo pienso de él, sino lo\nque él piensa de sí mismo . Herir a un hombre en su dignidad es un\ncrim

In [27]:
print(docs[0].page_content)

01
Cómo puede 
ayudarte la Inteligencia Artiﬁcial 
a crear contenido
La inteligencia artiﬁcial puede ayudarte a crear contenido 
proporcionando texto e imágenes a medida de tus 
indicaciones, ahorrándote tiempo y esfuerzo para que puedas 
centrarte en la producción de contenido de calidad. Por 
ejemplo, la inteligencia artiﬁcial puede proporcionar textos 
alrededor de una palabra clave, puede plantear la mejor 
manera de estructurar un texos para mantener a los lectores 
interesados, e incluso puede crear piezas de contenido por sí 
misma.
En esta guía te explicaré como crear contenido a partir de 2 
herramientas: Stable Diffusion y ChatGPT (GPT3).
Stable Diffusion es un modelo Open Source de generación de 
imágenes con Inteligencia Artiﬁcial.
Por otro lado ChatGPT, es una herramienta basada en GPT 3.5, 
que permite crear textos a partir de establecer un diálogo con 
la Inteligencia Artiﬁcial.
Guia Inteligencia Artiﬁcial - 03


In [28]:
docs = docsearch.similarity_search_with_score(query, k = 3)
docs

[(Document(page_content='01\nCómo puede \nayudarte la Inteligencia Artiﬁcial \na crear contenido\nLa inteligencia artiﬁcial puede ayudarte a crear contenido \nproporcionando texto e imágenes a medida de tus \nindicaciones, ahorrándote tiempo y esfuerzo para que puedas \ncentrarte en la producción de contenido de calidad. Por \nejemplo, la inteligencia artiﬁcial puede proporcionar textos \nalrededor de una palabra clave, puede plantear la mejor \nmanera de estructurar un texos para mantener a los lectores \ninteresados, e incluso puede crear piezas de contenido por sí \nmisma.\nEn esta guía te explicaré como crear contenido a partir de 2 \nherramientas: Stable Diffusion y ChatGPT (GPT3).\nStable Diffusion es un modelo Open Source de generación de \nimágenes con Inteligencia Artiﬁcial.\nPor otro lado ChatGPT, es una herramienta basada en GPT 3.5, \nque permite crear textos a partir de establecer un diálogo con \nla Inteligencia Artiﬁcial.\nGuia Inteligencia Artiﬁcial - 03', metadata={'pa

## haciendo loader masivo

In [ ]:


loader = PyPDFLoader("pdf/Como_ganar_amigos_e_influir_sobre_los_demas.pdf")
data = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(documents = data)


embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_documents(docs, embeddings, index_name = index_name)
index = pc.Index(index_name)
print(index.describe_index_stats())

In [85]:
from langchain_pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
import time

list_loader = [
    # 'pdf/El_hombre_en_busca_de_sentido_Viktor_Frankl.pdf',
    # 'pdf/El_poder_del_ahora_Eckhart_Tolle.pdf',
    # 'pdf/No_me_puedes_lastimar.pdf',
    # 'pdf/El_pequeño_libro_de_la_influecia-.pdf',
    # 'pdf/El_sutil_arte_de_que_te_importe_una_m.pdf',
    # 'pdf/Habitos_atomicos.pdf',
    # 'pdf/Habitos_para_ser_millonario.pdf',
    # 'pdf/La_semana_laboral_4_horas.pdf',
    # 'pdf/Las_48_leyes_del_poder_Robert_Greene.pdf',
    # 'pdf/Piense_y_hagase_rico.pdf',
    'pdf/Rompe_la_barrera_del_no_Chris_Voss.pdf'
    #
    # 'pdf/Como_ganar_amigos_e_influir_sobre_los_demas.pdf'
]

for load_pdf in list_loader:
    print(load_pdf)
    loader = PyPDFLoader(load_pdf)
    print(loader)
    print('cargando libro')
    data_book = loader.load()
    print('comenzando split...')
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 300, 
        chunk_overlap = 20
    )
    docs = text_splitter.split_documents(documents = data_book)
    print(docs)
    print(len(docs))
    print('Se realizo el split correctamente..')
    print('Iniciando proceso de embedding')
    embeddings = OpenAIEmbeddings()
    docsearch = Pinecone.from_documents(docs, embeddings, index_name = index_name)
    index = pc.Index(index_name)
    print(index.describe_index_stats())
    time.sleep(40)


pdf/Rompe_la_barrera_del_no_Chris_Voss.pdf
cargando libro
comenzando split...
[Document(page_content='R o m p e  l a  b a r r e r a  d e l  n o\n \n9 principios para negociar\ncomo si te fuera la vida en ello\n \n \nC HRIS  V OSS\ncon T AHL R AZ\n \n \n \nT raducción de María Serrano Giménez', metadata={'source': 'pdf/Rompe_la_barrera_del_no_Chris_Voss.pdf', 'page': 1}), Document(page_content='SÍGUENOS EN\n \n @Ebooks\n \n @megustaleer\n \n @megustaleer', metadata={'source': 'pdf/Rompe_la_barrera_del_no_Chris_Voss.pdf', 'page': 2}), Document(page_content='Para mi madr e y mi padr e,\nque me demostrar on lo que es el amor incondicional\ny me enseñar on los valor es del trabajo dur o y la integridad', metadata={'source': 'pdf/Rompe_la_barrera_del_no_Chris_Voss.pdf', 'page': 3}), Document(page_content='1\nLas nuevas reglas\n \n \n \nE staba intimidado.\nLlevaba más de veinte años en el FBI, quince de ellos ejerciendo de\nnegociador en crisis con rehenes, desde Nueva Y ork hasta Filipinas 

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for text-embedding-ada-002 in organization org-KrYlfON8JcUwAWnYYriyFsaw on tokens per min (TPM): Limit 150000, Requested 154537. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
docs

[Document(page_content='El poder del ahora es un libro para releer una y otra vez, y cada vez que lo\nhaga usted logrará profundizar más y encontrar nuevos significados.\nMuchas persona s querrán estudiarlo toda la vida, pues es una guía, un curso\ncompleto de meditación y realización. Es un libro con el poder de cambiar\nvidas, de despertarnos para comprender plenamente quiénes somos. El\nmensaje de Tolle es el mismo que Cristo y Buda enseñaron: se puede\nalcanzar un estado de iluminación, de realización espiritual, aquí y ahora.\nEs posible liberarnos del sufrimiento, de la ansiedad y la neurosis de la vida\ndiaria. Para lograrlo sólo tenemos que llegar a comprender que la causa de\nnuestros problem as no son los demás, ni «el mundo de allá afuera», sino\nnuestra propia mente, aparentemente incapaz de concentrarse en el ahora\npor estar siempre pensando en el pasado y preocupándose por el futuro.\nComo dice Marc Allen, el editor de la edición norteameri cana, en el\nPrefacio, «sin es

In [71]:
query = "Como ser millonario"
docs = docsearch.similarity_search(query, k = 5)
docs

[Document(page_content='HABIT OS P ARA SER MILLONARIO', metadata={'page': 6.0, 'source': 'pdf/Habitos_para_ser_millonario.pdf'}),
 Document(page_content='su patrimonio neto se incrementa entre un ocho y un diez por ciento al año\nsobre la cantidad de dinero que han trabajado. No buscan esquemas para\nhacerse ricos rápidamente ni para ganar dinero fácil. Son pacientes,\npersistentes y previsores. Se imponen la disciplina de ahorrar y acumular\ndinero año tras año. No especulan, ni toman riesgos, ni buscan la manera\nrápida de ganar dinero rápida y fácilmente. Gracias a su manera habitual de\npensar sobre su dinero, su riqueza va aumentando cada año. Al final,\nsobrepasan la marca del millón de dólares y, normalmente, siguen\nsobrepasándola.\nDesarrolla la mentalidad de millonario\nEl filóso fo empresarial Jim Rohn escribió: «Llegar a ser millonario no es\ntan difícil, pero no es lo más importante. La parte más importante de hacerte\nmillonario es la persona en la que te has convertido p

In [76]:
from langchain.vectorstores import Pinecone

text_field = "text"
index = pc.Index(index_name)

embed = OpenAIEmbeddings()

vectorstore = Pinecone(
    index, 
    embed.embed_query, 
    text_field
)

/Users/eaariash/desarrollo/ia_generativa/chat_ia_generative/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/Users/eaariash/desarrollo/ia_generativa/chat_ia_generative/venv/lib/python3.11/site-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [79]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

template = """Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
{context}
Pregunta: {question}
Respuesta: 
"""

prompt = PromptTemplate(
    template = template, input_variables = ["context", "question"]
)

llm = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0.0
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore.as_retriever(search_kwargs={'k': 3 }), #Por defecto recupera los 4 documentos más relevantes as_retriever(search_kwargs={'k': 3 })
    verbose = True,
    chain_type_kwargs = {"prompt": prompt, "verbose": True}
)

In [84]:
qa.run('En que consiste la teoria de la cuña')



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
sujetos a la ley de Parkinson: «Los gastos aumentan hasta  cubrir los
ingresos».
Ahorra la mitad de tus incrementos futuros
Así es cómo funciona la teoría de la cuña. Cuando le pides a alguien que
ahorre una parte de sus ingresos actuales, casi siempre responde que es una
buena idea, pero que no pued e. Ahorrar parte de sus ingresos actuales,
supondría reducir su nivel de vida: mudarse a una casa más pequeña,
conducir un coche más pequeño, comer comida más barata y salir menos.
Puesto que el ser humano es una criatura de costumbres, aunque
racionalmente afirme que es buena idea ahorrar , la renuncia al nivel de vida
que eso conlleva ría le parece tan i

'La teoría de la cuña consiste en ahorrar el 50 % de cada incremento que recibas en tu nómina a partir de hoy, sin reducir tu nivel de vida actual.'